In [30]:
import pandas as pd
import numpy as np

# Step 1: Load your dataset 
df = pd.read_csv('customer_data.csv')
df.head()


,CustomerID,Name,Age,Gender,Country,SignupDate,LastLogin,TotalPurchase,PreferredDevice,Email
0,e3e70682-c209-4cac-a29f-6fbed82c07cd,Christopher Williams,63.0,Male,USA,2021-12-22,2023-09-12,2141.15,dasktop,margaret03@bullock.info
1,f728b4fa-4248-4e3a-8a5d-2f346baa9455,Kevin Hopkins,42.0,Female,UK,2023-04-21,2024-06-28,2863.67,NaN,tammy76@mcintyre.org
2,eb1167b3-67a9-4378-bc65-c1e582e2e662,Sonya Stafford,76.0,Femlae,UK,2020-09-16,2023-06-13,2427.18,NaN,lauramichael@hotmail.com
3,f7c1bd87-4da5-4709-9471-3d60c8a70639,Matthew Schmidt,53.0,Unknown,Indai,2021-05-16,2024-12-15,5986.07,moblie,@example.com
4,e443df78-9558-467f-9ba9-1faf7a024204,Kristen Banks,74.0,NaN,UK,2022-04-14,2025-01-08,3374.72,dasktop,blacknicole@smith-lewis.com


In [31]:
# Step 2: Clean column names
df.columns = df.columns.str.strip().str.lower()

In [32]:
df['gender'].value_counts(dropna=False)

gender
NaN        14635
Femlae      7347
Female      7319
Male        7238
mle         7234
Unknown     7227
Name: count, dtype: int64

In [33]:
# Fix inconsistent text and typos
# 1. Fill actual np.nan values first
df['gender'] = df['gender'].fillna('unknown')

# 2. Now that all values are strings, clean them
df['gender'] = df['gender'].astype(str).str.strip().str.lower()

# 3. Fix typos and remaining inconsistencies
df['gender'] = df['gender'].replace({
    'mle': 'male',
    'femlae': 'female',
    'f': 'female',
    'm': 'male',
    'balck': 'unknown',
    'unkown': 'unknown',
    'nan': 'unknown' # This will catch any that were already strings
})

# Final check
print(df['gender'].value_counts(dropna=False))


gender
unknown    21862
female     14666
male       14472
Name: count, dtype: int64


In [34]:
df['preferreddevice'].value_counts(dropna=False)

preferreddevice
NaN        17122
dasktop     8571
mobile      8512
desktop     8468
moblie      8327
Name: count, dtype: int64

In [35]:
# Fix typos and remaining inconsistencies
df['preferreddevice'] = df['preferreddevice'].fillna('unknown')

df['preferreddevice'] = df['preferreddevice'].replace({
    'dasktop' : 'desktop',
    'moblie' : 'mobile',
    'Unknown': 'unknown'
})

# Final check
print(df['preferreddevice'].value_counts(dropna=False))

preferreddevice
unknown    17122
desktop    17039
mobile     16839
Name: count, dtype: int64


In [36]:
df['country'].value_counts(dropna=False)

country
NaN        12670
Indai       6504
India       6422
USA         6405
Germany     6400
UK          6334
Canda       6265
Name: count, dtype: int64

In [37]:
df['country'] = df['country'].fillna('unknown')

df['country'] = df['country'].replace({
    'Indai' : 'India',
    'Canda' : 'Canada',
})

# Final check
print(df['country'].value_counts(dropna=False))

country
India      12926
unknown    12670
USA         6405
Germany     6400
UK          6334
Canada      6265
Name: count, dtype: int64


In [38]:
# Step 4: Fix names (remove digits)
if 'name' in df.columns:
    df['name'] = df['name'].astype(str).str.replace(r'\d+', '', regex=True).str.strip()

In [39]:
# Step 5: Fix age decimals (round off)
# Fill missing ages with mean age, then round and convert
# Fix invalid ages
df['age'] = pd.to_numeric(df['age'], errors='coerce')  # ensure numeric
df.loc[(df['age'] < 0) | (df['age'] > 100), 'age'] = np.nan  # invalid ages → NaN
df['age'] = df['age'].round()  # round decimals
df['age'] = df['age'].fillna(df['age'].median()) #filling nan values with the median value

In [40]:
# Step 6: Convert date columns
for col in ['signupdate', 'lastlogin']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

In [41]:
# Convert to numeric (in case some are strings)
df['totalpurchase'] = pd.to_numeric(df['totalpurchase'], errors='coerce')

# Replace negative values with NaN (invalid)
df.loc[df['totalpurchase'] < 0, 'totalpurchase'] = np.nan

# Fill missing or invalid (NaN) purchase amounts with 0
df['totalpurchase'] = df['totalpurchase'].fillna(0)

In [42]:
email_pattern = r'^[^@]+@[^@]+\.[^@]+$'
df['email'] = df['email'].astype(str).str.strip()

df.loc[
    ~df['email'].str.match(email_pattern, na=False),
    'email'
] = np.nan
df['email'] = df['email'].fillna('unknown')

In [43]:
# Step 7: Remove duplicates
df = df.drop_duplicates()

In [44]:
# Step 8: Display summary
print("\n Cleaned Data Overview:")
print(df.info())
display(df.head(10))


 Cleaned Data Overview:
<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   customerid       49511 non-null  object        
 1   name             50000 non-null  object        
 2   age              50000 non-null  float64       
 3   gender           50000 non-null  object        
 4   country          50000 non-null  object        
 5   signupdate       48976 non-null  datetime64[ns]
 6   lastlogin        48963 non-null  datetime64[ns]
 7   totalpurchase    50000 non-null  float64       
 8   preferreddevice  50000 non-null  object        
 9   email            50000 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(6)
memory usage: 4.2+ MB
None


,customerid,name,age,gender,country,signupdate,lastlogin,totalpurchase,preferreddevice,email
0,e3e70682-c209-4cac-a29f-6fbed82c07cd,Christopher Williams,63.0,male,USA,2021-12-22,2023-09-12,2141.15,desktop,margaret03@bullock.info
1,f728b4fa-4248-4e3a-8a5d-2f346baa9455,Kevin Hopkins,42.0,female,UK,2023-04-21,2024-06-28,2863.67,unknown,tammy76@mcintyre.org
2,eb1167b3-67a9-4378-bc65-c1e582e2e662,Sonya Stafford,76.0,female,UK,2020-09-16,2023-06-13,2427.18,unknown,lauramichael@hotmail.com
3,f7c1bd87-4da5-4709-9471-3d60c8a70639,Matthew Schmidt,53.0,unknown,India,2021-05-16,2024-12-15,5986.07,mobile,unknown
4,e443df78-9558-467f-9ba9-1faf7a024204,Kristen Banks,74.0,unknown,UK,2022-04-14,2025-01-08,3374.72,desktop,blacknicole@smith-lewis.com
5,23a7711a-8133-4876-b7eb-dcd9e87a1613,Andrew Adams,74.0,unknown,USA,2022-03-09,2023-07-13,740.75,mobile,james83@henderson.com
6,1846d424-c17c-4279-a3c6-612f48268673,,51.0,female,UK,2021-01-16,2024-09-23,7702.33,desktop,woodsusan@yahoo.com
7,fcbd04c3-4021-4ef7-8ca5-a5a19e4d6e3c,Amanda Webster,38.0,unknown,USA,2020-11-26,2023-11-21,5103.92,mobile,jason95@collins.com
8,b4862b21-fb97-4435-8856-1712e8e5216a,David Holt,63.0,unknown,Germany,2021-01-07,2024-04-16,1666.40,mobile,tammythomas@brown.org
9,259f4329-e6f4-490b-9a16-4106cf6a659e,Cameron Guerrero,29.0,male,UK,2023-11-14,2024-12-15,7062.40,desktop,murraydylan@gmail.com


In [45]:
# Step 9: Save cleaned dataset
df.to_csv('cleaned_customer_data.csv', index=False)
print("\n Cleaned dataset saved as 'cleaned_customer_data.csv'")



 Cleaned dataset saved as 'cleaned_customer_data.csv'
